Vocab restriction approaches
- Grammar and allowed words in context. -- Will not scale to 1000s of words. -- Might work well with a better model (something to try!!!)
- Finetune model with allowed words in context
- Finetune model with allowed current next tokens in context (?)
- Embedding Space Regularization -- add input of allowed next tok -- add term to loss that penalizes that token's embedding from getting to close to any non allowed

- without tuning -- small vocab can use first approach, but large vocab can generate one token at a time and, if one is invalid given current word, "undo" it and regenerate with added prompt "Do not use the word XYZ"
  - Say something -> I
  - Say something -> I am
  - Say something -> I am
  - Say something -> I am a teach
  - Say something -> I am a teacher <space, meaning end of word>
  - ERR: can't say "teacher"
  - Say something. Do not use the word "teacher" -> I am a
  - Say something. Do not use the word "teacher" -> I am a plumb
  - Say something. Do not use the word "teacher" -> I am a plumber
    - or go back a few if different word changes more stuff. Maybe even redo whole sentence.
  - ...
  - Say something -> I am a teacher <space, meaning end of word>
  - ERR: can't say "teacher"
  - Say something. Do not use the word "teacher" -> I am
  - Say something. Do not use the word "teacher" -> I am
  - Say something. Do not use the word "teacher" -> I am an <"an" instead of "a">
  - Say something. Do not use the word "teacher" -> I am an act
  - Say something. Do not use the word "teacher" -> I am an actor
    

In [ ]:
srun -G1 --pty bash -c "source /data/ai_club/team_3_2024-25/team3-env-finetune/bin/activate; \
    hostname; \
    jupyter notebook \
        --ServerApp.root_dir=$(pwd) \
        --ServerApp.password='' \
        --ServerApp.open_browser=False \
        --ServerApp.allow_origin='*' \
        --ServerApp.allow_remote_access=True \
        --ServerApp.port=14321 \
        --ServerApp.ip='*'
"

In [1]:
# import torch
# import torch.nn as nn
import tensorflow as tf

import transformers

2025-01-28 15:45:21.987047: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-28 15:45:22.000926: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738097122.017315 3876104 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738097122.022226 3876104 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-28 15:45:22.038486: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
MODEL_NAME = 'distilbert/distilgpt2'
# MODEL_NAME = 'gpt2'

In [3]:
model = transformers.AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# class ModifiedTransformerBlock(nn.Module):
#     def __init__(self, original_layer, residual_dim):
#         super().__init__()
        
#         self.original_layer = original_layer
#         for param in self.original_layer.parameters():
#             param.requires_grad = False
        
#         self.residual_transform = nn.Linear(residual_dim, residual_dim)

#     def forward(self, hidden_states, attention_mask=None, **kwargs):
#         print(kwargs)
#         attn_output = self.original_layer.attention(hidden_states, attention_mask=attention_mask)
#         ffn_output = self.original_layer.feed_forward(attn_output)
        
#         transformed_residual = self.residual_transform(hidden_states)
#         output = ffn_output + transformed_residual
        
#         return output

# original_layer = model.transformer.h[0]
# residual_dim = original_layer.attn.c_attn.weight.size(0)

# modified_layer = ModifiedTransformerBlock(original_layer, residual_dim)

# model.transformer.h[0] = modified_layer

In [4]:
model.hidden_states

AttributeError: 'GPT2LMHeadModel' object has no attribute 'hidden_states'

In [5]:
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)

In [6]:
tokens = tokenizer('human: hi\nbot: I AM A ROBOT\nhuman: huh?\n', return_tensors='tf')
out_ids = model.generate(**tokens, max_length=100, pad_token_id=tokenizer.eos_token_id, temperature=0.8, do_sample=True)
print(tokenizer.decode(out_ids[0]))

I0000 00:00:1738097131.399919 3876104 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13764 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:db:00.0, compute capability: 7.5


RuntimeError: Invalid device string: '/job:localhost/replica:0/task:0/device:GPU:0'